https://towardsdatascience.com/linear-regression-and-gradient-descent-using-only-numpy-53104a834f75

Research Linear regression using numpy

# Libs

In [1]:

import numpy as np

Create a random dataset with 5 features, and create randomly m and q that we will have to discover. 

In [2]:
X = np.random.rand(10000,5)
m = np.random.randint(low = 1, high = 20,size = (5,1))  #parametri random tra low e high
q = np.random.rand(1)
# "@" symbol represents the matrix multiplication operation
y = (X @ m) + q 

noise = np.random.randn(y.shape[0], y.shape[1])
y = y + noise

# X.shape, m.shape, q.shape, y.shape

We then need to add a feature of “1” concatenating it with the dataset we already have and also add q to the vector m.

In [3]:
X = np.concatenate([X , np.ones((X.shape[0],1))], axis = 1)
m = np.concatenate([m,q.reshape(1,-1)],axis = 0)

In [4]:
def partial_derivative(X_batch, y_batch, m_stat):
    y_pred = X_batch @ m_stat
    n = len(X_batch)
    
    df_dm =  (-2/n) * (X_batch.T @ (y_batch - y_pred))
    df_dm = df_dm.reshape(len(df_dm),-1)

    return df_dm

In [5]:
def mean_squared_error(X,y,m_stat):
    y_pred = X @ m_stat
    mse = np.sum(((y_pred - y)**2),axis = 0) / len(X)
  
    return mse

In [6]:
def training(X, y, batch_size, lr, epochs):
    
    
    for epoch in range(epochs):
        

        #random initial statistics
        if epoch == 0:
            m_stat = np.random.rand(X.shape[1],1)
  
        #shuffle X and y using same permutation
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
  
        X = X[indices]
        y = y[indices]
  
        #store comulative derivative
        cumulative_derivative = np.zeros((X.shape[1],1))
  
        for batch in range(len(X)//batch_size):
            start = batch*batch_size
            stop = (batch*batch_size) + batch_size
    
            X_batch = X[start:stop]
            y_batch = y[start:stop]
            
            #derivative
            cumulative_derivative = cumulative_derivative + partial_derivative(X_batch, y_batch, m_stat)
    
            #updating rule
            m_stat = m_stat - (lr*cumulative_derivative)
      
    print(f"epoch: {epoch} ----> MSE: {mean_squared_error(X,y,m_stat)}")
        
    return m_stat

In [7]:
batch_size = 1024
lr = 0.01
epochs = 5000

m_stat = training(X,y, batch_size,lr,epochs)

epoch: 4999 ----> MSE: [0.99882239]


In [8]:
print(m_stat,"\n")
print(m)

X_test = np.random.rand(500,5)
X_test = np.concatenate([X_test,np.ones(shape = (500,1))] , axis = 1)
y_test = X_test @ m

y_preds = X_test @ m_stat
mse = mean_squared_error(X_test, y_test, m_stat)

print("mse" , mse)

print(y_test[:5])
print(y_preds[:5])

[[14.99695402]
 [11.01957488]
 [17.95893943]
 [ 9.03028122]
 [ 6.03396655]
 [ 0.90775832]] 

[[15.        ]
 [11.        ]
 [18.        ]
 [ 9.        ]
 [ 6.        ]
 [ 0.93691247]]
mse [0.00043447]
[[38.94083709]
 [27.7264576 ]
 [34.57887695]
 [31.82001168]
 [33.24099434]]
[[38.92518548]
 [27.68951496]
 [34.55670131]
 [31.78643017]
 [33.20151036]]


# Research

In [9]:
import numpy as np

# Creating a random dataset
X = np.random.rand(10000, 5)
# Make y like linear regression formula y = y=mx+b
y = X @ np.random.rand(5, 1) + np.random.rand(1)
# Adding random noise to the y values
# np.random.randn(y.shape[0], y.shape[1]) generates random numbers from a standard normal distribution
# The shape of the noise matrix matches the shape of y, so it's added element-wise
# This step simulates real-world variability and noise in the data
y += np.random.randn(y.shape[0], y.shape[1])

In [10]:
def partial_derivative(X_batch, y_batch, m_stat):
    # Calculate predicted y values using the current coefficients
    y_pred = X_batch @ m_stat
    
    # Calculate the number of samples in the batch
    n = len(X_batch)
    
    # Compute the derivative of the Mean Squared Error (MSE) with respect to coefficients
    # This is a vectorized operation using matrix multiplication
    df_dm = (-2/n) * (X_batch.T @ (y_batch - y_pred))
    
    # Reshape the derivative vector into a column vector
    df_dm = df_dm.reshape(len(df_dm), -1)
    
    # Return the computed derivative
    return df_dm

In [11]:
def mean_squared_error(X, y, m_stat):
    # Calculate predicted y values using the current coefficients
    y_pred = X @ m_stat
    
    # Calculate the Mean Squared Error (MSE)
    mse = np.sum(((y_pred - y)**2), axis=0) / len(X)
    return mse

In [12]:
def training(X, y, batch_size, lr, epochs):
    '''
    Training loop
    '''
    for epoch in range(epochs):
        # Initialize coefficients with random values at the beginning of training
        if epoch == 0:
            m_stat = np.random.rand(X.shape[1], 1)
        # Create indices for shuffling the data
        indices = np.arange(y.shape[0])
        
        # Shuffle the indices to randomize the order of samples
        np.random.shuffle(indices)
        
        # Shuffle input features (X) and target values (y) based on the shuffled indices
        X = X[indices]
        y = y[indices]
        
        # Initialize a cumulative derivative vector with zeros
        cumulative_derivative = np.zeros((X.shape[1], 1))
        
        # Divide the data into batches and update coefficients iteratively
        for batch in range(len(X)//batch_size):
            start = batch * batch_size
            stop = (batch * batch_size) + batch_size
            X_batch = X[start:stop]
            y_batch = y[start:stop]
            
            # Compute the cumulative derivative for the batch using partial_derivative function
            cumulative_derivative += partial_derivative(X_batch, y_batch, m_stat)
            
            # Update coefficients using the gradient descent update rule
            m_stat = m_stat - (lr * cumulative_derivative)
    
    # Print epoch number and calculated Mean Squared Error (MSE) for monitoring
    print(f"epoch: {epoch} ----> MSE: {mean_squared_error(X, y, m_stat)}")
        
    # Return the final coefficients after training
    return m_stat


In [13]:
batch_size = 1024
lr = 0.01
epochs = 200
m_stat = training(X, y, batch_size, lr, epochs)
# Testing the model
X_test = np.random.rand(500, 5)
y_test = X_test @ np.random.rand(5, 1)
y_preds = X_test @ m_stat
mse = mean_squared_error(X_test, y_test, m_stat)

print("Model Parameters:\n", m_stat, "\n")
print("Mean Squared Error:", mse)
print("True Labels:\n", y_test[:5])
print("Predicted Labels:\n", y_preds[:5])

epoch: 199 ----> MSE: [1.03108303]
Model Parameters:
 [[0.80974045]
 [1.27188573]
 [0.37484911]
 [0.90413796]
 [0.54658935]] 

Mean Squared Error: [1.12423096]
True Labels:
 [[1.05659981]
 [1.54483571]
 [0.91242442]
 [1.34183903]
 [0.65907988]]
Predicted Labels:
 [[2.35768864]
 [2.84482871]
 [2.64846054]
 [2.75034266]
 [1.21605543]]
